In [1]:
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import random

In [13]:
def scrap_indeed(search,max_search = 100,save_csv = True):
    search_term = search.replace(' ','+')
    url = 'https://www.indeed.com/jobs?q={}&limit=50&radius=25&start=0'.format(search_term)
    browser = webdriver.Chrome('C:/Program Files (x86)/chromedriver.exe')
    browser.get(url)
    
    try:
        close_button = browser.find_element_by_id('prime-popover-close-button')
        close_button.click()
    except (ElementNotVisibleException, NoSuchElementException):
        print('No Popup')
        
    max_rating = browser.find_element_by_class_name('ratings').size['width']
    data = pd.DataFrame(columns = ['Job_Title','Company','Estimated_Salary','Date_Posted','Link_Job_Company','Link_Job_Indeed','Location','Rating_In_Stars','Job_Description'])

    x = 0
    y = 0
    for j in range(max_search // 50):
        #z = 0
        job_elements =  browser.find_elements_by_xpath("//div[@class='row result clickcard']")
        try:
            job_elements.append(browser.find_element_by_xpath("//div[@class='lastRow row result clickcard']"))
        except Exception:
            pass

        for i in range(len(job_elements)):

            redirect_to_comp_site = job_elements[i].find_element_by_tag_name('a').get_attribute('href')

            try:
                estimated_salary = job_elements[i].find_element_by_class_name('no-wrap').text
            except NoSuchElementException:
                estimated_salary = 'NA'

            date = job_elements[i].find_element_by_class_name('date').text   
            job_elements[i].click()
    #        browser.implicitly_wait(3)

            browser.switch_to_window(browser.window_handles[1])
            sleep(random.randint(0,3))
            indeed_link = browser.current_url
            retry_count = 0
            while True:
                try:
                    retry_count += 1
                    title =  browser.find_element_by_class_name('jobtitle').text
                    post = browser.find_element_by_id('job_summary').text
                    company = browser.find_element_by_class_name('company').text
                    location = browser.find_element_by_class_name('location').text
                    break
                except NoSuchElementException:
                    if retry_count > 5:
                        print('Unable to fetch data. Skipping Page.....')
                        break
                    else:
                        print('Unable to fetch data. Retrying....')





            #Can be replaced by  browser.find_element_by_class_name('rating')[0].size['width']
            #But it rounds that up
            try:
                rtng_style_str = browser.find_element_by_class_name('rating').get_attribute('style')
                rtng_style_int = float(rtng_style_str.split(' ')[1][:-3])
                rating = (rtng_style_int*5)/max_rating
            except NoSuchElementException:
                rating = 'NA'


            print('Completed Post {} of Page {} - {}'.format(i+1,j+1,title))

            data = data.append({'Job_Title':title,'Company':company,'Link_Job_Company':redirect_to_comp_site,'Estimated_Salary':estimated_salary,'Date_Posted':date,'Link_Job_Indeed':indeed_link,'Location':location,'Rating_In_Stars':rating,'Job_Description':post},ignore_index=True)    
            browser.close()
            browser.switch_to_window(browser.window_handles[0])






        y += 50

        url = url.replace('start=' + str(x),'start=' +str(x+50))
        x+= 50

        browser.get(url)
        print('Moving on to page ' + str(j+2))
        sleep(2)
        try:
            #Only on 2nd page
            browser.find_element_by_id('popover-x-button').click()
        except:
            print('No Newsletter Popup Found')
            
    if save_csv:
        data.to_csv(search + '.csv')
        

    
    

In [ ]:
scrap_indeed('machine learning',1500) #27th Jan
scrap_indeed('artificial intelligence',1500) #27th Jan
scrap_indeed('data scientist',1500) #28th Jan